# Discretization of errors

So far we've considered $X$ errors and $Z$ errors in the context of the 9-qubit Shor code, and in this section we'll consider arbitrary errors.
What we'll find is that, to handle such errors, we don't need to do anything different from what we've already discussed;
the ability to correct $X$ errors, $Z$ errors, or both, implies the ability to correct arbitrary errors.
This phenomenon is sometimes called the *discretization of errors*.

## Unitary qubit errors

Let's begin with single-qubit *unitary* errors.
For example, such an error could correspond to a very small rotation of the Bloch sphere, possibly representing an error incurred by a gate that isn't perfect, for instance.
Or it could be any other unitary operation on a qubit and not necessarily one that's close to the identity.

It might seem like correcting for such errors is difficult.
After all, there are infinitely many possible errors like this, and it's inconceivable that we could somehow identify each error exactly and then undo it.
However, as long as we can correct for a bit-flip, a phase-flip, or both, then we will succeed in correcting an arbitrary single-qubit unitary error using the procedures described earlier in the lesson.

To see why this is the case, let us recognize first that we can express an arbitrary $2 \times 2$ unitary matrix $U,$ representing an error on a single qubit, as a linear combination of the four Pauli matrices (including the identity matrix).

$$
U = \alpha \mathbb{I} + \beta X + \gamma Y + \delta Z
$$

As we will see, when the error detection circuits are run, the measurements that give us the syndrome bits effectively collapse the state of the encoding probabilistically to one where an error (or lack of an error) represented by one of the four Pauli matrices has taken place.
(It follows from the fact that $U$ is unitary that the numbers $\alpha,$ $\beta,$ $\gamma,$ and $\delta$ must satisfy $\vert\alpha\vert^2 + \vert\beta\vert^2 + \vert\gamma\vert^2 + \vert\delta\vert^2 = 1,$
and indeed, the values $\vert\alpha\vert^2,$ $\vert\beta\vert^2,$ $\vert\gamma\vert^2,$ and $\vert\delta\vert^2$ are the probabilities with which the encoded state collapses to one for which the corresponding Pauli error has occurred.)

To explain how this works in greater detail, it will be convenient to use subscripts to indicate which qubit a given qubit unitary operation acts upon.
For example, using Qiskit's qubit numbering convention $(\mathsf{Q}_8,\mathsf{Q}_7,\ldots,\mathsf{Q}_0)$ to number the 9 qubits used for the Shor code, we have these expressions for various unitary operations on single qubits, where in each case we tensor the unitary matrix with the identity matrix on every other qubit.

$$
\begin{aligned}
X_0 & = \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes X \\[1.5mm]
Z_4 & = \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes Z \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \\[1.5mm]
U_7 & = \mathbb{I} \otimes U \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I}
\end{aligned}
$$

So, in particular, for a given qubit unitary operation $U,$ we can specify the action of $U$ applied to qubit $k$ by the following formula, which is similar to the one before except that each matrix represents an operation applied to qubit $k.$

$$
U_k = \alpha \mathbb{I}_k + \beta X_k + \gamma Y_k + \delta Z_k
$$

Now suppose that $\vert\psi\rangle$ is the 9-qubit encoding of a qubit state.
If the error $U$ takes place on qubit $k,$ we obtain the state $U_k \vert\psi\rangle,$ which can be expressed as a linear combination of Pauli operations acting on $\vert\psi\rangle$ as follows.

$$
U_k \vert\psi\rangle = \alpha \vert\psi\rangle + \beta X_k\vert\psi\rangle + \gamma Y_k\vert\psi\rangle + \delta Z_k\vert\psi\rangle
$$

At this point let's make the substitution $Y = iXZ.$

$$
U_k \vert\psi\rangle = \alpha \vert\psi\rangle + \beta X_k\vert\psi\rangle + i \gamma X_kZ_k\vert\psi\rangle + \delta Z_k\vert\psi\rangle
$$

Now consider the error-detection and correction steps described previously.
We can think about the measurement outcomes for the three inner code parity checks along with the one for the outer code collectively as a single syndrome consisting of 8 bits.
Just prior to the actual standard basis measurements that produce these syndrome bits, the state has the following form.

$$
\begin{gathered}
\alpha\,\vert \mathbb{I} \text{ syndrome}\rangle \otimes \vert\psi\rangle \\
+ \beta\,\vert X_k \text{ syndrome}\rangle \otimes X_k\vert\psi\rangle \\
+ i \gamma\,\vert X_k Z_k \text{ syndrome}\rangle \otimes X_k Z_k\vert\psi\rangle \\
+ \delta\,\vert Z_k \text{ syndrome}\rangle \otimes Z_k\vert\psi\rangle
\end{gathered}
$$

To be clear, we have two systems at this point.
The system on the left is the 8 qubits we'll measure to get the syndrome, where $\vert \mathbb{I} \text{ syndrome}\rangle,$
$\vert X_k \text{ syndrome}\rangle,$ and so on, refer to whatever 8-qubit standard basis state is consistent with the corresponding error (or non-error).
The system on the right is the 9 qubits we're using for the encoding.

Notice that these two systems are now correlated (in general), and this is the key to why this works.
By measuring the syndrome, the state of the 9 qubits on the right effectively collapses to one in which a Pauli error consistent with the measured syndrome has been applied to one of the qubits.
Moreover, the syndrome itself provides enough information so that we can undo the error and recover the original encoding $\vert\psi\rangle.$

In particular, if the syndrome qubits are measured and the appropriate corrections are made, we obtain a state that can be expressed as a density matrix,

$$
\xi \otimes \vert\psi\rangle\langle\psi\vert,
$$

where

$$
\begin{aligned}
\xi = & \vert\alpha\vert^2 \vert \mathbb{I} \text{ syndrome}\rangle\langle \mathbb{I} \text{ syndrome}\vert \\[1mm]
& + \vert\beta\vert^2 \vert X_k \text{ syndrome}\rangle\langle X_k \text{ syndrome}\vert\\[1mm]
& + \vert\gamma\vert^2 \vert X_k Z_k \text{ syndrome}\rangle\langle X_k Z_k \text{ syndrome}\vert\\[1mm]
& + \vert\delta\vert^2 \vert Z_k \text{ syndrome}\rangle\langle Z_k \text{ syndrome}\vert.
\end{aligned}
$$

Critically, this is a product state:
we have our original, uncorrupted encoding as the right-hand tensor factor,
and on the left we have a density matrix $\xi$ that describes a random error syndrome.
There is no longer any correlation with the system on the right, which is the one we care about, because the errors have been corrected.
At this point we can throw the syndrome qubits away or reset them so we can use them again.
This is how the randomness — or *entropy* — created by errors is removed from the system.

This is the discretization of errors for the special case of unitary errors.
In essence, by measuring the syndrome, we effectively *project* the error onto an error that's described by a Pauli matrix.

At first glance it may seem too good to be true that we can correct for arbitrary unitary errors like this, even errors that are tiny and hardly noticeable on their own.
But, what's important to realize here is that this is a unitary error on a *single* qubit, and by the design of the code, a single-qubit operation can't change the state of the logical qubit that's been encoded.
All it can possibly do is to move the state out of the subspace of valid encodings, but then the error detections collapse the state and the corrections bring it back to where it started.

## Arbitrary qubit errors

Finally, let's consider arbitrary errors that are not necessarily unitary.
To be precise, we'll consider an error described by an arbitrary qubit channel $\Phi.$
For example, this could be a dephasing or depolarizing channel, a reset channel, or a strange channel that we've never thought about before.

The first step is to consider any Kraus representation of $\Phi.$

$$
\Phi(\sigma) = \sum_j A_j \sigma A_j^{\dagger}
$$

This is a qubit channel, so each $A_j$ is a $2\times 2$ matrix, which we can express as a linear combination of Pauli matrices.

$$
A_j = \alpha_j \mathbb{I} + \beta_j X + \gamma_j Y + \delta_j Z
$$

This allows us to express the action of the error $\Phi$ on a chosen qubit $k$ in terms of Pauli matrices as follows.

$$
\Phi_k \bigl( \vert\psi\rangle\langle\psi\vert\bigr) =
\sum_j (\alpha_j \mathbb{I}_k + \beta_j X_k + \gamma_j Y_k + \delta_j Z_k) \vert\psi\rangle\langle\psi\vert
(\alpha_j \mathbb{I}_k + \beta_j X_k + \gamma_j Y_k + \delta_j Z_k)^{\dagger}
$$

In short, we've simply expanded out all of our Kraus matrices as linear combinations of Pauli matrices.

If we now compute and measure the error syndrome, and correct for any errors that are revealed, we'll obtain a similar sort of state to what we had in the case of a unitary error:

$$
\xi \otimes \vert\psi\rangle\langle\psi\vert,
$$

where this time we have

$$
\begin{aligned}
\xi = & \sum_j \Bigl(\vert\alpha_j\vert^2 \vert \mathbb{I} \text{ syndrome}\rangle\langle \mathbb{I} \text{ syndrome}\vert \\[-3mm]
& \qquad + \vert\beta_j\vert^2 \vert X_k \text{ syndrome}\rangle\langle X_k \text{ syndrome}\vert\\[2mm]
& \qquad + \vert\gamma_j\vert^2 \vert X_k Z_k \text{ syndrome}\rangle\langle X_k Z_k \text{ syndrome}\vert\\[2mm]
& \qquad + \vert\delta_j\vert^2 \vert Z_k \text{ syndrome}\rangle\langle Z_k \text{ syndrome}\vert \Bigr).
\end{aligned}
$$

The details are a bit messier and are not shown here.
Conceptually speaking, the idea is identical to the unitary case.

## Generalization

The discretization of errors generalizes to other quantum error-correcting codes, including ones that can detect and correct errors on multiple qubits.
In such cases, errors on multiple qubits can be expressed as *tensor products* of Pauli matrices, and correspondingly different syndromes specify Pauli operation corrections that might be performed on multiple qubits rather than just one qubit.

Again, by measuring the syndrome, errors are effectively projected or collapsed onto a discrete set of possibilities represented by tensor products of Pauli matrices, and by correcting for those Pauli errors, we can recover the original encoded state.
Meanwhile, whatever randomness is generated in the process is moved into the syndrome qubits, which are discarded or reset, thereby removing the randomness generated in this process from the system that stores the encoding.